In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data_file ='SpamData/02_Training/train_data.txt'
test_data_file ='SpamData/02_Training/test_data.txt'
vocab_size = 2500

In [3]:
#Read and Load features from .txt files into numpy array
sparse_train_data = np.loadtxt(train_data_file,delimiter=' ',dtype=int)
sparse_test_data = np.loadtxt(test_data_file,delimiter=' ',dtype=int)

In [4]:
sparse_train_data[:5] #first 5 rows

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [5]:
sparse_train_data[-5:] #last 5 rows

array([[5798, 2035,    0,    2],
       [5798, 2040,    0,    1],
       [5798, 2272,    0,    1],
       [5798, 2342,    0,    1],
       [5798, 2370,    0,    1]])

In [6]:
print(sparse_train_data.shape[0],'==> No of rows in TRAIN file')
print(sparse_test_data.shape[0],'==> No of rows in TEST file')

263535 ==> No of rows in TRAIN file
112601 ==> No of rows in TEST file


In [7]:
print(np.unique(sparse_train_data[:,0]).size,'==> No of emails in TRAIN file')
print(np.unique(sparse_test_data[:,0]).size,'==> No of emails in TEST file')
#[:,0] ==> Notation for selecting all the rows in the 0th column.

4012 ==> No of emails in TRAIN file
1725 ==> No of emails in TEST file


# Creating a full Matrix

In [8]:
column_names=['DOC_ID']+['CATEGORY']+list(range(0,vocab_size))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [9]:
len(column_names)

2502

In [10]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5796, 5797, 5798])

In [11]:
full_train_data = pd.DataFrame(index=index_names,columns=column_names)
full_train_data.fillna(value=0,inplace=True)
#fillna fills the na values with 0s

In [12]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Creating a Full Matrix from Sparse Matrix

In [13]:
def make_full_matrix(sparse_matrix,no_words,doc_idx=0,word_idx=1,cat_idx=2,freq_idx=3):
    #no_words - size of vocabulary
    #doc_idx=0 - position of document id in sparse matrix set to 1st column
    #word_idx=1 - position of word id in sparse matrix set to 2nd column 
    #cat_idx=2 - position of label/category (spam -> 1 , non-spam -> 0) set to 3rd column
    #freq_idx=3 - position of occurrence of word  id in sparse matrix set to 1st column
    column_names = column_names=['DOC_ID']+['CATEGORY']+list(range(0,vocab_size))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index=index_names,columns=column_names)
    full_matrix.fillna(value=0,inplace=True)
    for i in range(sparse_matrix.shape[0]):
        doc_no = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_no,'DOC_ID'] = doc_no
        full_matrix.at[doc_no,'CATEGORY'] = label
        full_matrix.at[doc_no,word_id] = occurence
    full_matrix.set_index('DOC_ID',inplace=True)
    return full_matrix

In [14]:
%%time
full_train_data = make_full_matrix(sparse_train_data,vocab_size)

CPU times: total: 33.1 s
Wall time: 33.8 s


In [15]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


 # Counting the tokens to train the Naive Bayes Model


In [16]:
#calculating token Probabilities
# P(Spam | Viagra) =   ( P(Viagra | Spam) * P(Spam) ) / P(Viagra)

### Calculating the probability of spam - the percent of spam messages in the training dataset & storing it to var prob_spam

In [20]:
full_train_data.CATEGORY.size

4012

In [28]:
full_train_data.CATEGORY.sum()

1246

In [34]:
full_train_data.CATEGORY.tail()

DOC_ID
5792    0
5795    0
5796    0
5797    0
5798    0
Name: CATEGORY, dtype: int64

In [36]:
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size

In [39]:
print('probability of spam is', prob_spam)

probability of spam is 0.31056829511465606


### Total Number of Words/Tokens

In [43]:
full_train_features = full_train_data.loc[:,full_train_data.columns !='CATEGORY']

In [44]:
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4012,)

In [52]:
email_lengths[0:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [54]:
total_wc = email_lengths.sum()
total_wc

447628

### Number of token in Spam & Ham Emails

In [55]:
#Creating a subset of the email_lengths series that only contains the spam messages.
spam_lengths = email_lengths[full_train_data.CATEGORY==1]
spam_lengths.shape

(1246,)

In [58]:
ham_lengths = email_lengths [full_train_data.CATEGORY==0]
ham_lengths.shape

(2766,)

In [60]:
email_lengths.shape[0]-spam_lengths.shape[0]-ham_lengths.shape[0]
#Verifying

0

In [70]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

254877

In [71]:
spam_wc = spam_lengths.sum()
spam_wc

192751

In [73]:
spam_wc + nonspam_wc - total_wc

0

In [82]:
print("average number of words in spam emails  {:.2f}".format(spam_wc/spam_lengths.shape[0]))
print("average number of words in ham emails  {:.2f}".format(nonspam_wc/ham_lengths.shape[0]))

average number of words in spam emails  154.70
average number of words in ham emails  92.15


### Summing the tokens Occuring in Spam

In [83]:
full_train_features.shape

(4012, 2500)

In [87]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
train_spam_tokens.shape

(1246, 2500)

In [92]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [93]:
summed_spam_tokens.tail()

2495     4
2496     2
2497    17
2498    18
2499     2
dtype: int64

In [95]:
#Repeating same process for ham messages..  and storing them in summed_ham_tokens

### Summing the tokens Occuring in ham

In [97]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY==0]
summed_ham_tokens = train_ham_tokens.sum(axis=0)+1

In [98]:
summed_ham_tokens.tail()

2495    33
2496    34
2497    17
2498    12
2499    31
dtype: int64

In [101]:
train_ham_tokens[2499].sum()+1

31